In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')


forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [3]:
# aoi
from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ AOI loaded: 1 features
   Area: 144217.67 hectares


In [ ]:
pl_dir_apr27 = forestry.config.get('pl_dir_apr27')
pl_dir_jun8 = forestry.config.get('pl_dir_jun8')
pl_dir_jul1 = forestry.config.get('pl_dir_jul1')
pl_dir_jul30 = forestry.config.get('pl_dir_jul30')
pl_dir_sep21_2024 = forestry.config.get('pl_dir_sep21_2024')

In [ ]:
# pl_dir_jul1

In [ ]:
sep_21_2024_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep21_2024, "composite_file_format.tif"))
apr_27_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_apr27, "composite_file_format.tif"))
jun_8_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jun8, "composite_file_format.tif"))
jul_1_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul1, "composite_file_format.tif"))
jul_30_img = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul30,"composite_file_format.tif"))


In [ ]:
apr_27_img.bandNames().getInfo()

In [ ]:
# from wmts_manager import WMTSManager

# wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)
# wmts.addLayer(jul_30_img, {'bands': ['B5', 'B7', 'B3'],
#    'min': 0,
#    'max': 6000,
#    'gamma': 1.5}, 'jul_30_img'
#              )

# wmts.publish()

In [ ]:
## cloud masking start
udm_sep21_2024 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_sep21_2024, "composite_udm2_file_format.tif"))
udm_apr27 =  ee.Image.loadGeoTIFF(os.path.join(pl_dir_apr27, "composite_udm2_file_format.tif"))
udm_jun8 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jun8, "composite_udm2_file_format.tif"))
udm_jul1 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul1, "composite_udm2_file_format.tif"))
udm_jul30 = ee.Image.loadGeoTIFF(os.path.join(pl_dir_jul30, "composite_udm2_file_format.tif"))


In [ ]:
udm_apr27.bandNames().getInfo()

In [ ]:
udm_jun8.bandNames().getInfo()

In [ ]:
### two band to check in UDM
# condition no cloud

confidence_threshold = 40  # >70 is clear
#using other


def cloud_masking(img, udm_img, confidence_threshold):
    """
    Apply cloud masking using UDM with explicit reprojection.
    """
    # Create masks
    clear_val = 1
    no_snow_val = 0
    no_shadow_val = 0
    no_haze_light_val = 0
    no_haze_heavy_val = 0
    no_cloud_val = 0

    clear_mask = udm_img.select('clear').eq(clear_val)
    no_cloud = udm_img.select('snow').eq(no_snow_val)
    no_shadow = udm_img.select('shadow').eq(no_shadow_val)
    no_haze_light = udm_img.select('haze_light').eq(no_haze_light_val)
    no_haze_heavy = udm_img.select('haze_heavy').eq(no_haze_heavy_val)
    no_cloud = udm_img.select('cloud').eq(no_cloud_val)
    confidence_mask = udm_img.select('confidence').gt(confidence_threshold)
    
    # combined_mask = clear_mask.And(confidence_mask)
    combined_mask = clear_mask.And(confidence_mask.And(no_shadow.And(no_haze_light.And(no_haze_heavy.And(no_cloud.And(no_cloud))))))
    
    # Reproject UDM mask to match image's first band projection
    # This is critical for proper alignment
    img_first_band = img.select(0)
    combined_mask = combined_mask.reproject(
        crs=img_first_band.projection().crs(),
        scale=img_first_band.projection().nominalScale()
    )
    
    # Apply mask
    cloudless_img = img.updateMask(combined_mask)
    
    return cloudless_img

sep_21_2024_img_cloudless = cloud_masking(sep_21_2024_img, udm_sep21_2024, confidence_threshold)
apr_27_img_cloudless = cloud_masking(apr_27_img, udm_apr27, confidence_threshold)
jun_8_img_cloudless = cloud_masking(jun_8_img, udm_jun8, confidence_threshold)
jul_1_img_cloudless = cloud_masking(jul_1_img, udm_jul1, confidence_threshold)
jul_30_img_cloudless = cloud_masking(jul_30_img, udm_jul30, confidence_threshold)

In [ ]:
jul_1_img_cloudless.propertyNames().getInfo()

In [ ]:
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)

wmts.addLayer(sep_21_2024_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'sep_21_2024_img_cloudless'
             )

wmts.addLayer(apr_27_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'apr_27_img_cloudless'
             )

wmts.addLayer(jun_8_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jun_8_img_cloudless'
             )

wmts.addLayer(jul_1_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jul_1_img_cloudless'
             )

wmts.addLayer(jul_30_img_cloudless, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'jul_30_img_cloudless'
             )

wmts.publish()

In [ ]:
### based on the images converted to the gee object, we will merge them into gee image collection 
sep_21_2024_img_cloudless = sep_21_2024_img_cloudless.set('system:time_start', ee.Date.fromYMD(2024, 9, 21).millis())
apr_27_img_cloudless = apr_27_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 4, 27).millis())
jun_8_img_cloudless = jun_8_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 6, 8).millis())
jul_1_img_cloudless= jul_1_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 7, 1).millis())
jul_30_img_cloudless = jul_30_img_cloudless.set('system:time_start', ee.Date.fromYMD(2025, 7, 30).millis())

list_ee_image = [sep_21_2024_img_cloudless, apr_27_img_cloudless, jun_8_img_cloudless, jul_1_img_cloudless, jul_30_img_cloudless]
planet_scope_col= ee.ImageCollection(list_ee_image)

In [ ]:
planet_scope_col.first().get('system:band_names').getInfo()

In [ ]:
planet_scope_col.first().get('system:time_start')

In [ ]:
from forestry_carbon_arr.utils import fill_temporal_gaps_linear

# Use on your SG-smoothed collection
bands_to_fill = ['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']  # or the bands you smoothed
planet_scope_col_interpolated = fill_temporal_gaps_linear(planet_scope_col, bands_to_fill)

print('Original monthly (with NaNs):', planet_scope_col.size().getInfo())
print('Filled monthly collection:', planet_scope_col_interpolated.size().getInfo())

In [ ]:
recent_img = planet_scope_col_interpolated.filter(
    ee.Filter.eq('system:time_start', ee.Date.fromYMD(2025, 7, 30).millis())
).first()

wmts.addLayer(recent_img, {'bands': ['B5', 'B7', 'B3'],
   'min': 0,
   'max': 6000,
   'gamma': 1.5}, 'recent_img_FIX'
             )

wmts.publish()

In [ ]:
import ee

# Example: Export single image to GCS
def export_image_to_gcs(image, gcs_bucket, gcs_path, scale=10, crs='EPSG:4326', region=None, max_pixels=1e13):
    """
    Export Earth Engine Image to Google Cloud Storage
    
    Parameters:
    -----------
    image : ee.Image
        Earth Engine Image to export
    gcs_bucket : str
        GCS bucket name (e.g., 'my-bucket' or 'gs://my-bucket')
    gcs_path : str
        Path within bucket (e.g., 'exports/fcd_2020.tif')
    scale : float
        Pixel scale in meters (default: 30)
    crs : str
        Coordinate reference system (default: 'EPSG:4326')
    region : ee.Geometry, optional
        Region to export (default: None, uses image bounds)
    max_pixels : int
        Maximum pixels to export (default: 1e9)
    
    Returns:
    --------
    ee.batch.Task : Export task
    """
    # Clean bucket name (remove gs:// if present)
    if gcs_bucket.startswith('gs://'):
        gcs_bucket = gcs_bucket.replace('gs://', '').split('/')[0]
    
    # Full GCS path
    gcs_uri = f"gs://{gcs_bucket}/{gcs_path}"
    
    # Export parameters
    export_params = {
        'image': image,
        'description': gcs_path.split('/')[-1].replace('.tif', ''),  # Task name
        'bucket': gcs_bucket,
        'fileNamePrefix': gcs_path.replace('.tif', ''),  # Path without extension
        'scale': scale,
        'crs': crs,
        'maxPixels': max_pixels,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {
            'cloudOptimized': True  # COG format
        }
    }
    
    # Add region if provided
    if region is not None:
        export_params['region'] = region
    
    # Create export task
    task = ee.batch.Export.image.toCloudStorage(**export_params)
    
    # Start the task
    task.start()
    
    print(f"✅ Export task started: {gcs_uri}")
    print(f"   Task ID: {task.id}")
    
    return task

In [ ]:
task = export_image_to_gcs(
        image=recent_img,
        gcs_bucket='remote_sensing_saas',
        gcs_path=f'01-korindo/planet_merged/planet_recent_2025_07_interpolated_2024_9_2025_7.tif',
        scale=10,
        crs='EPSG:32749',  # UTM zone for your AOI
        region=aoi_ee.geometry()  # Optional: clip to AOI
    )

In [4]:
## after exported -->
input_image = ee.Image.loadGeoTIFF(os.path.join(f'gs://remote_sensing_saas',f'01-korindo/planet_merged/planet_recent_2025_07_interpolated_2024_9_2025_7.tif'))
input_image.propertyNames().getInfo()    

['system:bands', 'system:band_names']

In [5]:
input_image.bandNames().getInfo()

['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

In [6]:
input_image_fix = input_image.select(['B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']). \
                                rename(['coastal_blue','blue','green1','green','yellow','red','redEdge','nir'])
input_image_fix.bandNames().getInfo()

['coastal_blue', 'blue', 'green1', 'green', 'yellow', 'red', 'redEdge', 'nir']

In [7]:
# forestry.config

In [8]:
from wfs_manager import WFSManager

AOI = ee.FeatureCollection('projects/remote-sensing-476412/assets/korindo_with_buffer')

wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
wfs.addLayer(AOI, "AOI Boundary")
wfs.publish()

{'status': 'success',
 'total_layers': 1,
 'successful_layers': 1,
 'failed_layers': 0,
 'layers': {'aoi_boundary': {'status': 'success',
   'layer_name': 'aoi_boundary',
   'fc_url': 'http://fastapi:8000/fc/aoi_boundary',
   'wfs_url': 'http://localhost:8001/wfs?service=WFS&version=1.1.0&request=GetFeature&typename=aoi_boundary',
   'feature_count': 1,
   'response': {'message': "FeatureCollection 'aoi_boundary' created successfully",
    'name': 'aoi_boundary',
    'type': 'FeatureCollection',
    'count': 1}}},
 'service_urls': {'wfs_capabilities': 'http://localhost:8001/wfs?service=WFS&version=1.1.0&request=GetCapabilities',
  'wfs_base': 'http://localhost:8001/wfs'}}

In [9]:
# Set I_satellite to 'Custom' in your config
forestry.config['I_satellite'] = 'Custom'

# Run eligibility with your custom image
el = forestry.run_eligibility(
    config=forestry.config,
    use_gee=True,
    custom_image=input_image_fix  # Your pre-processed image
)

processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
no thermal band, choosing Planet images
Processing means center of AVI_BSI please wait
Now we proceed to the PCA of Vegetation density
Success get the PCA normalized of VD => SVI
Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2
finish processing PCA, the result: FCD1_1 and FCD2_1 please continue
snic list bands: ['red_mean', 'green_mean', 'blue_mean', 'nir_mean', 'ndwi_mean', 'msavi2_mean', 'MTVI2_mean', 'NDVI_mean', 'VARI_mean', 'FCD1_1_mean', 'FCD2_1_mean', 'area', 'clusters_min', 'width', 'height']
Adding the map of Forest, FCD >= 45% and mask only if not water in area (hansen) and NDWI
Adding the map of Shrubland, FCD <  45% and FCD >= 45%
Adding the map of Grassland or Openland, FCD  < 45%
Processing - the zoning classification
finish processing, merging all the zone into one image


In [10]:
gbm = el['intermediate_results']['classifier_results']['classified_image_gbm']
gbm.propertyNames().getInfo()

['system:bands', 'system:band_names']

In [11]:
# Assess accuracy for all algorithms
accuracy_results = forestry.assess_accuracy(
    eligibility_results=el,
    output_dir='./01_output'
)

# # Or assess only specific algorithms
# accuracy_results = forestry.assess_accuracy(
#     eligibility_results=el,
#     algorithms=['fcd', 'gbm'],
#     output_dir='./01_output'
# )

# # Or use custom validation points
# accuracy_results = forestry.assess_accuracy(
#     eligibility_results=el,
#     validation_points=my_validation_points,
#     output_dir='./outputs',
#     label_column='type'  # Use your custom label column
# )

-------------------------------------------------------------------------------------
Algorithm of ML used: fcd
Confusion Matrix:
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 45, 0, 4, 0, 0, 0, 0, 0, 0], [0, 317, 0, 40, 5, 0, 0, 0, 0, 0], [0, 150, 0, 140, 19, 0, 0, 0, 0, 0], [0, 8, 0, 20, 51, 0, 0, 0, 0, 0], [0, 2, 0, 3, 2, 1, 0, 0, 0, 0], [0, 3, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 10, 0, 0, 0, 0, 0], [0, 11, 0, 16, 5, 0, 0, 0, 0, 0], [0, 2, 0, 0, 1, 0, 0, 0, 0, 0]]
Overall Accuracy: 0.2765460910151692
Producer's Accuracy: [[0], [0.9183673469387755], [0], [0.45307443365695793], [0.6455696202531646], [0.125], [0], [0], [0], [0]]
User's Accuracy: [[0, 0.08348794063079777, 0, 0.625, 0.5483870967741935, 1, 0, 0, 0, 0]]
Kappa: 0.1585625218736688
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
Algorithm of ML used: rf
Confusion Matrix:
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 8, 

In [12]:
from wmts_manager import WMTSManager

wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry(), clear_cache_first=True)

## LC
rf = el['intermediate_results']['classifier_results']['classified_image_rf']
# svm = el['intermediate_results']['classifier_results']['classified_image_svm']
# gbm = el['intermediate_results']['classifier_results']['classified_image_gbm']
# cart = el['intermediate_results']['classifier_results']['classified_image_cart']

rf_layer_name = el['intermediate_results']['layer_names']['land_cover_rf']
# svm_layer_name = el['intermediate_results']['layer_names']['land_cover_svm']
# gbm_layer_name = el['intermediate_results']['layer_names']['land_cover_gbm']
# cart_layer_name = el['intermediate_results']['layer_names']['land_cover_cart']

vis_param_lc = el['visualization_params']['land_cover']

wmts.addLayer(rf, vis_param_lc, rf_layer_name)
# wmts.addLayer(svm, vis_param_lc, svm_layer_name)
# wmts.addLayer(gbm, vis_param_lc, gbm_layer_name)
# wmts.addLayer(cart, vis_param_lc, cart_layer_name)
wmts.publish()

INFO:wmts_manager:WMTSManager initialized for project: korindo
INFO:wmts_manager:Added layer: Random_forest_lc_result
INFO:wmts_manager:Generating map IDs for 1 layers...


Generating GEE Map IDs...


INFO:wmts_manager:AOI processed: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
INFO:wmts_manager:Publishing 1 layers to WMTS...
INFO:gee_integration:GEE Integration Manager initialized:
INFO:gee_integration:  FastAPI URL: http://fastapi:8000
INFO:gee_integration:  MapStore Config: /usr/src/app/mapstore/configs/localConfig.json
INFO:gee_integration:Processing GEE analysis: korindo
INFO:gee_integration:🧹 Clearing duplicate projects before processing new analysis...


✅ Centroid calculated successfully with error margin 1
Calculated bbox from coordinates: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}
✅ AOI processed successfully:
   - Center: [111.90816469614359, -0.3125187262245936]
   - Area: Unknown
   - BBox: {'minx': 111.706493374869, 'miny': -0.457839253478653, 'maxx': 112.109836017418, 'maxy': -0.167195667545911}


INFO:cache_manager:No existing catalog entries to check for duplicates
INFO:gee_integration:✅ Duplicate clearing successful: 0 duplicates cleared, 0 unique projects kept
INFO:gee_integration:✅ Cache cleared: 0 duplicate entries, kept 0 unique projects
INFO:gee_integration:Using complex layer info for 'Random_forest_lc_result': ['tile_url', 'name', 'description', 'vis_params']
INFO:gee_integration:Registering with FastAPI: korindo_20251120_080023
INFO:gee_integration:✅ FastAPI registration successful: MapStore catalog updated successfully
INFO:gee_integration:Creating FastAPI proxy URLs for project: korindo_20251120_080023
INFO:gee_integration:✅ Created 1 FastAPI proxy URLs
INFO:gee_integration:Updating MapStore WMTS: korindo_20251120_080023
INFO:gee_integration:🔄 Forcing comprehensive WMTS refresh...
INFO:gee_utils:Starting comprehensive WMTS refresh...
INFO:gee_utils:Clearing old WMTS services...
ERROR:gee_utils:Error clearing WMTS services: 'list' object has no attribute 'get'
INFO:g

{'status': 'success',
 'project_id': 'korindo_20251120_080023',
 'project_name': 'korindo',
 'fastapi_registration': {'status': 'success',
  'message': 'MapStore catalog updated successfully',
  'layers_count': 1},
 'proxy_urls_creation': {'status': 'success',
  'message': 'Created 1 proxy URLs',
  'proxy_urls': {'Random_forest_lc_result': {'proxy_url': 'http://fastapi:8000/tiles/korindo_20251120_080023/Random_forest_lc_result/{z}/{x}/{y}',
    'original_url': 'https://earthengine.googleapis.com/v1/projects/remote-sensing-476412/maps/e97e87f4c0fa8630db223eb2eec518bc-63beeadbdbc26d05d4dadd156d92fad0/tiles/{z}/{x}/{y}',
    'layer_name': 'Random_forest_lc_result',
    'description': 'RANDOM_FOREST_LC_RESULT visualization from GEE analysis'}},
  'layers_count': 1},
 'wmts_configuration': {'status': 'success',
  'message': 'WMTS configuration updated successfully',
  'service_id': 'GEE_analysis_WMTS_layers',
  'layers_available': ['korindo_20251120_080023_Random_forest_lc_result'],
  'laye

In [ ]:
### using the GEE only, is only gets to 73% max. in parallel -->
## Training data should be improved e.g create area polygon for the Forest only,
##### Integrate to python ML project, 
## ds for pixel after scaled with indices (rbgnir si etc) without normalization
## ds normalized result (all bands)
## ds normalized result with the SNIC cluster based (OBIA)
## prioritize to create new OBIA in GEE based on arcgis